In [1]:
import torch
import torch.nn as nn
from torchcrf import CRF
import pandas as pd

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
a = torch.rand(32, 8)
b = torch.zeros([32, 8], dtype=torch.float)
b[0, 0] = 1
loss = nn.BCEWithLogitsLoss(reduction='none')(a, b)

In [ ]:
# loss = torch.where(b != 0, loss, torch.tensor([0.]))
loss

In [33]:
loss = torch.where(b != 0, loss, loss*0.5)

In [37]:
loss.shape

torch.Size([32, 8])

In [44]:
num_tags = 5  # number of tags is 5
model = CRF(num_tags)

In [48]:
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags) #n_seq x batch x logits
tags = torch.tensor([
  [0, 1], [2, 4], [3, 1]
], dtype=torch.long)  #nseq x batch
loss = model(emissions, tags)



In [57]:
a = torch.rand(30, 32, 142)
b = torch.zeros(30, 32, dtype=torch.long)
model = CRF(a.shape[-1])


In [58]:
model(a, b)

tensor(-4717.1206, grad_fn=<SumBackward0>)

In [5]:
list = []
name = ['train', 'test', 'dev']
for i in name:
    for label in open("/home/xps/educate/code/hust/XQA/docs/src/JointIDSF/PhoATIS/word-level" + f'/{i}/label'):
        if label.strip() not in list:
            list.append(label.strip())

In [7]:
len(list)

28

In [9]:
train = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_train.csv", header=None)


In [14]:
a = {'vietanh': 'ngu'}
a.update({'vie anh': 'ocho'})

(array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
        'atis_ground_service', 'atis_airline', 'atis_abbreviation',
        'atis_quantity'], dtype=object),
 4834)

In [15]:
test = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_test.csv", header=None)
test[0].unique(), len(test)

(array(['atis_flight', 'atis_airfare', 'atis_ground_service',
        'atis_airline', 'atis_flight_time', 'atis_quantity',
        'atis_abbreviation', 'atis_aircraft'], dtype=object),
 800)

In [7]:
lengths = {}
with open("/home/xps/educate/code/hust/XQA/data/raw/PhoATIS/word-level/train/seq.in", 'r') as f:
    for line in f.readlines():
        if len(line.strip().split(' ')) not in lengths.keys():
            lengths[len(line.strip().split(' '))] = 1
        else:
            lengths[len(line.strip().split(' '))] += 1 
        if len(line.strip().split(' ')) == 50:
            print(line.strip()) 
    

tôi muốn một chuyến bay khứ_hồi từ đồng_hới đến côn_đảo khởi_hành vào thứ tư ngày 16 tháng 6 đến côn_đảo vào buổi tối khoảng 7 giờ và trở về đồng_hới vào buổi tối ngày hôm sau khoảng 7 giờ hãng hàng_không nào có chuyến bay phù_hợp với tuyến đường đó


In [ ]:
sorted(lengths, key = lambda d: lengths[d])

In [1]:
import os 

In [2]:
os.path.basename('/hoa/hoctro')

'hoctro'

In [2]:
import pandas as pd
os.environ['dir'] = "/home/xps/educate/code/hust/XQA/data"
from utils.preprocess import string2list, get_label
from collections import Counter
from transformers import AutoTokenizer
import torch

data_dir = os.environ['dir']
raw_dir = data_dir + '/data/raw/PhoATIS'
processed_dir = data_dir + '/data/processed/PhoATIS'
qa_processed = data_dir + '/data/processed/QA'
tokenizer = AutoTokenizer.from_pretrained('NlpHUST/bert-base-vn')

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/train.csv")

In [4]:
test_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/test.csv")

In [5]:
dev_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/dev.csv")

In [173]:
def get_label(off_set_map, text, start, end=False):
    '''
      if END_IDX is provided, just use it and leave TEXT=''
    '''
    start_idx, end_idx = [], 0
    for i in range(len(off_set_map)):
        if off_set_map[i][0] == start:
            start_idx.append(i)
        elif off_set_map[i][0] == start + 1:
            start_idx.append(i)
        
        if len(text.split()) == 1 and start_idx != 0:
            end_idx = start_idx
            break

        if text != '' and start_idx != 0 and i > start_idx:
            if off_set_map[i][1] == start + len(text):
              end_idx = i 

            elif off_set_map[i][1] == start + len(text) + 1:
              end_idx = i

    # print("what", start_idx, end_idx)
    return torch.tensor(start_idx, dtype=torch.long), torch.tensor(end_idx, dtype=torch.long)

In [159]:
run = True
for i in range(len(dev_df)):
    q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
    input = tokenizer(q.strip(), c, return_tensors='pt',
                max_length=500,
                truncation="only_second",
                return_offsets_mapping=True,
                padding="max_length",)
    a, t = string2list(a, 'int'), string2list(t)
    start, end = 0, 0
    for i in range(len(a)):
        s, e = get_label(input['offset_mapping'][0], t[0], a[0])
        if e < s:
            print("???", s, e)
            run = False
            break
        start += s 
        end += e
    # if start == 0 or end == 0:
    #     q, c, a, t = q, c, a, t
    #     run = False
    #     break
    if not run:
        break
        
    




??? tensor(499) tensor(14)


In [160]:
q, c, t[0], a[0], c[a[0]:]

('Bản đồ địa chất được bắt đầu vẽ ở đâu?',
 'Bồn Paris tạo nên một tập hợp các lớp trầm tích kế tiếp. Đây là một trong những nơi đầu tiên trở thành đối tượng của việc xây dựng bản đồ địa chất và cho phép tạo ra nhiều lý thuyết về địa chất, như cổ sinh vật học và giải phẫu so sánh của Georges Cuvier. Bồn Paris được tạo trong khoảng thời gian 41 ngàn năm. Đây là một bồn thềm lục địa trên những khối núi từ thời Đại cổ sinh như khối núi Vosges, khối núi Massif central, khối núi Armorica. Với sự tạo thành của dãy Alps, bồn Paris bị đóng lại, chỉ còn mở ra phía biển Manche và Đại Tây Dương. Nó báo trước cho sự hình thành lưu vực sông Loire và sông Seine. Cuối thế Oligocen, bồn Paris trở thành lục địa.',
 'Bồn Paris',
 0,
 'Bồn Paris tạo nên một tập hợp các lớp trầm tích kế tiếp. Đây là một trong những nơi đầu tiên trở thành đối tượng của việc xây dựng bản đồ địa chất và cho phép tạo ra nhiều lý thuyết về địa chất, như cổ sinh vật học và giải phẫu so sánh của Georges Cuvier. Bồn Paris được tạ

In [178]:
input = tokenizer(q.strip(), c, return_tensors='pt',
                max_length=386,
                truncation="only_second",
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length",)
# a, t = string2list(a, 'int'), string2list(t)

In [180]:
a[0], t[0], q, c 

(0,
 'Bồn Paris',
 'Bản đồ địa chất được bắt đầu vẽ ở đâu?',
 'Bồn Paris tạo nên một tập hợp các lớp trầm tích kế tiếp. Đây là một trong những nơi đầu tiên trở thành đối tượng của việc xây dựng bản đồ địa chất và cho phép tạo ra nhiều lý thuyết về địa chất, như cổ sinh vật học và giải phẫu so sánh của Georges Cuvier. Bồn Paris được tạo trong khoảng thời gian 41 ngàn năm. Đây là một bồn thềm lục địa trên những khối núi từ thời Đại cổ sinh như khối núi Vosges, khối núi Massif central, khối núi Armorica. Với sự tạo thành của dãy Alps, bồn Paris bị đóng lại, chỉ còn mở ra phía biển Manche và Đại Tây Dương. Nó báo trước cho sự hình thành lưu vực sông Loire và sông Seine. Cuối thế Oligocen, bồn Paris trở thành lục địa.')

In [184]:
input['overflow_to_sample_mapping']


tensor([0])

In [186]:
sequence_ids = input.sequence_ids()
idx = 0
while sequence_ids[idx] != 1:
        idx += 1
context_start = idx
while sequence_ids[idx] == 1:
        idx += 1
context_end = idx - 1

In [ ]:
start_positions = []
end_positions = []
if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
    start_positions.append(0)
    end_positions.append(0)
else:
    # Otherwise it's the start and end token positions
    idx = context_start
    while idx <= context_end and offset[idx][0] <= start_char:
        idx += 1
    start_positions.append(idx - 1)
    idx = context_end
    while idx >= context_start and offset[idx][1] >= end_char:
        idx -= 1
    end_positions.append(idx + 1)

start_positions, end_positions

In [174]:
start, end = get_label(input['offset_mapping'][0], t[2], a[2])
start, end

0
1
12
13
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385


(tensor(385), tensor(14))

In [166]:
tokenizer.decode(input.input_ids[0][start.item():end.item()+1])

''

In [125]:
length = []

for i in range(len(dev_df)):
    q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
    a, c = string2list(a, 'int'), string2list(c)
    length.append(len(q.strip().split()))

In [108]:
sum(length)/len(length)

13.979868708971553

In [39]:
a, c

([432, 432, 432, 432],
 ['Năm 1871, Đức trở thành một quốc gia dân tộc khi hầu hết các quốc gia Đức thống nhất trong Đế quốc Đức do Phổ chi phối. Sau Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919, Đế quốc này bị thay thế bằng Cộng hòa Weimar theo chế độ nghị viện. Chế độ độc tài quốc xã được hình thành vào năm 1933, dẫn tới Chiến tranh thế giới thứ hai và một nạn diệt chủng. Sau một giai đoạn Đồng Minh chiếm đóng, hai nước Đức được thành lập: Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức. Năm 1990, quốc gia được tái thống nhất.'])

In [4]:
max = 0
for i in range(len(df)):
    item = df.iloc[i]
    start = string2list(item['start'], type='int')
    text = string2list(item['text'])
    break

In [24]:
a = "wefwefwefwfe@@@dfsdfsdf"
a.split("@@@")

['wefwefwefwfe', 'dfsdfsdf']

In [ ]:
from transformers import RobertaModel, AutoTokenizer
model_checkpoint = 'vinai/phobert-base'
# model = QAModule(model_checkpoint=model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
input = tokenizer("DIT CON ME", return_tensors='pt',
        max_length=258,
        truncation="only_second",
        padding="max_length",)
model(input_ids=input.input_ids.repeat(16, 1), attention_mask=input.attention_mask.repeat(16, 1))


In [1]:
model.device

NameError: name 'model' is not defined